In [3]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from impyute.imputation.cs import fast_knn
from impyute.imputation.cs import mice

In [4]:
import os
os.getcwd()

'C:\\Users\\payal'

In [5]:
alldata = pd.read_csv("C:\\Users\\payal\\Downloads\\Book2_new.csv", na_values=['na'])

In [6]:
gdp = pd.read_excel("C:\\Users\\payal\\Downloads\\gdp,gnp.xlsx")

In [7]:
gdp = gdp.drop(index = [0,1])

In [8]:
gdp.columns

Index(['Date', 'GDPACUR$', 'Unnamed: 2'], dtype='object')

In [9]:
gdp = gdp.rename(columns = {"Date":"Date", 'GDPACUR$':"GDP", 'Unnamed: 2': "GNP"})

In [10]:
alldata

,Year,Security,Revenue,Cost of Revenue,Gross Profit,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,...,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap
0,2018,AAPL,265595.000,163756.000,101839.000,30941.000,70898.000,70898.000,3.0025,81801.000,...,0.680,1.3297,10.0179,19.0155,13.3993,37.1708,3.1039,12.9369,5.6334,1.073391e+06
1,2019,AAPL,260174.000,161782.000,98392.000,34462.000,63930.000,63930.000,2.9900,76477.000,...,0.750,1.4349,10.7447,18.4110,14.0853,40.1345,3.1713,11.2844,5.0913,9.722690e+05
2,2020,AAPL,274515.000,169559.000,104956.000,38668.000,66288.000,66288.000,3.3100,78844.000,...,0.795,0.9743,29.1732,34.2703,15.8203,41.5230,3.8294,14.0611,3.8487,1.906151e+06
3,2021,AAPL,365817.000,212981.000,152836.000,43887.000,108949.000,108949.000,5.6700,121933.000,...,0.865,0.6114,38.2537,26.2512,21.9035,40.0303,4.4406,17.2563,3.8407,2.413423e+06
4,2022,AAPL,394328.000,223546.000,170782.000,51345.000,119437.000,119437.000,6.1500,132441.000,...,0.900,0.5488,47.3313,24.6203,24.3173,38.7899,3.7334,14.4808,3.1782,2.398369e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,2018,MPWR,582.382,259.714,322.668,209.180,113.488,113.488,2.4900,127.152,...,1.300,0.6395,7.7195,48.5501,13.7852,2.2041,13.0495,12.4906,15.0592,4.941206e+03
531,2019,MPWR,627.921,281.596,346.325,243.763,102.562,102.562,2.5200,119.667,...,1.600,0.9116,10.0383,73.6208,14.5470,2.1342,10.9196,11.6370,17.7341,7.764520e+03
532,2020,MPWR,844.452,378.498,465.954,307.072,158.882,158.882,3.6700,182.535,...,2.000,0.7127,17.1512,100.8583,18.8326,2.6602,12.4713,14.1275,21.3530,1.657813e+04
533,2021,MPWR,1207.798,522.339,685.459,423.042,262.417,262.417,5.2800,298.244,...,2.400,0.4882,18.3438,95.7422,26.3418,2.5084,10.3088,14.0723,26.8935,2.281947e+04


## Prepare Consolidated Dataset


In [11]:
alldata.sort_values(by=['Security', 'Year'], ascending = [True, True], inplace=True)
alldata.reset_index(drop=True, inplace=True)

In [12]:
##Merging gdp, gnp and consumer confidence data, sectors
cons = pd.read_excel("C:\\Users\\payal\\Downloads\\bpdata.xlsx", na_values=['na'])
cons

,Sm,Security Name,Sector,Average Volume 30 Day,Current Market Capitalization of a Share Class,Current Market Cap,EBITDA,Return on Assets,Price to Book Ratio,Beta,Dividend 12 Month Yield,Total Debt to Total Equity,Total Debt to Total Assets,EBITDA Growth 5 Year CAGR,Volatility 30 Day,Volatility 180 Day,Volatility 260 Day,Average Total Assets Growth
0,A,Agilent Technologies Inc,Health Care,3.936229e+05,4.235338e+04,4.235338e+10,1980.00,12.454109,7.558581,1.066537,0.596817,55.061263,27.734523,13.461198,24.524184,32.929508,35.115744,NaN
1,AAL,American Airlines Group Inc,Industrials,4.305724e+06,1.050106e+04,1.050106e+10,5591.00,0.193623,NaN,1.283619,NaN,NaN,67.505717,-1.180414,28.986561,46.575032,55.554577,NaN
2,AAP,Advance Auto Parts Inc,Consumer Discretionary,3.581420e+05,8.009666e+03,8.009666e+09,1561.91,4.145528,2.990106,1.060158,4.439840,151.819171,33.832426,13.769114,28.306072,37.138342,38.288729,NaN
3,AAPL,Apple Inc,Information Technology,1.821958e+07,2.433890e+06,2.433890e+12,132441.00,26.148106,42.960803,1.172139,0.597907,261.446164,37.555811,13.120696,26.240551,33.896968,35.703153,NaN
4,ABBV,AbbVie Inc,Health Care,1.656987e+06,2.747524e+05,2.747524e+11,26785.00,8.296242,15.922016,0.636607,3.677465,371.325273,46.245452,19.280246,16.253951,22.335803,23.373684,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,ZBH,Zimmer Biomet Holdings Inc,Health Care,4.289808e+05,2.617821e+04,2.617821e+10,1685.10,1.039477,2.1668,0.901307,0.770589,49.195976,28.086965,-1.976727,16.613423,25.580576,27.552415,NaN
500,ZBRA,Zebra Technologies Corp,Information Technology,1.574658e+05,1.572008e+04,1.572008e+10,781.00,6.737485,5.757184,1.462048,NaN,80.461032,29.207066,5.949530,38.955311,48.734365,47.704187,NaN
501,ZION,Zions Bancorp NA,Financials,6.362069e+05,7.248126e+03,7.248126e+09,NaN,0.992640,1.627859,1.054362,3.301887,227.447374,12.428388,NaN,25.502001,34.551551,37.958936,2.606497
502,ZTS,Zoetis Inc,Health Care,6.557376e+05,7.862282e+04,7.862282e+10,3433.00,14.667823,17.86477,1.017242,0.795522,184.714967,54.492462,11.999729,27.825315,32.944050,32.628422,NaN


In [13]:
cons.rename(columns={'Sm':'Security'}, inplace=True)


In [14]:
alldata = alldata.merge(cons[['Security', 'Sector']], on='Security', how='left')

In [15]:
## Missing values identification
x = alldata.apply(lambda s: 1 if (sum(s.isnull() * 1)) > 0.5 * len(s) else np.nan).dropna()
x


Series([], dtype: float64)

In [16]:
gdp["Year"] = gdp.Date.dt.year

In [17]:
gdp_df = gdp[["GDP", "Year", "GNP"]]

In [18]:
alldata1 = pd.merge(alldata, gdp_df, on='Year')

In [19]:
conf = pd.read_excel("C:\\Users\\payal\\Downloads\\confidence.xlsx")

In [20]:
alldata1 = pd.merge(alldata1, conf, on = "Year")

In [21]:
mktcap = pd.read_excel("C:\\Users\\payal\\Downloads\\marketcap.xlsx")

In [22]:
mktcap["Year"] = mktcap.Date.dt.year
mktcap

,Date,PX_LAST,PX_MID,Year
0,2022-12-31,4.106193e+07,4.106193e+07,2022
1,2021-12-31,5.376457e+07,5.376457e+07,2021
2,2020-12-31,4.264076e+07,4.264076e+07,2020
3,2019-12-31,3.437586e+07,3.437586e+07,2019
4,2018-12-31,2.687200e+07,2.687200e+07,2018
5,2017-12-31,2.964440e+07,2.964440e+07,2017
6,2016-12-31,2.520446e+07,2.520446e+07,2016
7,2015-12-31,2.354414e+07,2.354414e+07,2015
8,2014-12-31,2.441372e+07,2.441372e+07,2014
9,2013-12-31,2.228066e+07,2.228066e+07,2013


In [23]:
mktcap1 = mktcap[["Year", "PX_LAST"]]

In [24]:
alldata1 = pd.merge(alldata1, mktcap1, on = "Year")

In [25]:
alldata1 = alldata1.drop("Standard error", axis = 1)

In [26]:
alldata1

,Year,Security,Revenue,Cost of Revenue,Gross Profit,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,...,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,Index value,PX_LAST
0,2018,AAPL,265595.000,163756.000,101839.000,30941.000,70898.000,70898.000,3.0025,81801.000,...,37.1708,3.1039,12.9369,5.6334,1.073391e+06,Information Technology,20533.1,20823.36,70.1,2.687200e+07
1,2018,ACGL,5450.568,NaN,NaN,4608.796,962.256,NaN,1.7600,NaN,...,NaN,NaN,NaN,22.8925,1.075359e+04,Financials,20533.1,20823.36,70.1,2.687200e+07
2,2018,ADI,6224.689,1974.293,4250.396,2350.807,1899.589,1899.589,4.0500,2698.652,...,3.4711,8.0806,9.3698,30.4414,3.227051e+04,Information Technology,20533.1,20823.36,70.1,2.687200e+07
3,2018,AEP,16195.700,5786.500,10409.200,13513.000,2682.700,2682.700,3.9000,5083.100,...,6.1746,2.9213,8.4850,38.6578,3.686520e+04,Utilities,20533.1,20823.36,70.1,2.687200e+07
4,2018,ALL,39815.000,NaN,NaN,37193.000,2960.000,NaN,5.7800,NaN,...,NaN,NaN,NaN,58.1837,2.743316e+04,Financials,20533.1,20823.36,70.1,2.687200e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,2022,XEL,15310.000,6959.000,8351.000,12882.000,2428.000,2428.000,3.1800,5262.000,...,9.7057,4.4388,7.0262,30.3415,3.853091e+04,Utilities,25462.7,25640.13,63.1,4.106193e+07
536,2022,YUM,6842.000,1745.000,5097.000,4655.000,2187.000,2187.000,4.6300,2411.000,...,NaN,10.1807,11.0000,-31.7000,3.586240e+04,Consumer Discretionary,25462.7,25640.13,63.1,4.106193e+07
537,2022,ZBRA,5781.000,3157.000,2624.000,2095.000,529.000,529.000,8.8600,781.000,...,4.6736,4.6671,7.6066,53.1180,1.319268e+04,Information Technology,25462.7,25640.13,63.1,4.106193e+07
538,2022,ZTS,8080.000,2454.000,5626.000,NaN,2917.000,2917.000,4.5100,3433.000,...,1.1500,6.8395,6.8825,9.4975,6.797107e+04,Health Care,25462.7,25640.13,63.1,4.106193e+07


In [27]:
alldata1 = alldata1.rename(columns = {"Index value":"confid_index", 'PX_LAST':"Sp_mkt_cap"})

In [28]:
alldata1["Sector"].value_counts()

Information Technology    120
Industrials                75
Financials                 70
Health Care                45
Consumer Discretionary     45
Utilities                  35
Communication Services     35
Consumer Staples           30
Real Estate                25
Materials                  25
Energy                     25
Name: Sector, dtype: int64

In [29]:
## Missing values
## Since none of the columns have more than 60% of values as nan, we don't have to delete columns
## Counting number of na values in each column

na_counts = alldata1.isna().sum()
na_counts

Year                              0
Security                          0
Revenue                           0
Cost of Revenue                 110
Gross Profit                    110
Total Operating Expenses        241
Operating Income or Losses        0
EBIT                             35
Basic Earnings per Share          0
EBITDA                           35
Cash and Cash Equivalents         0
Accounts & Notes Receivable      25
Total Assets                      0
Total Current Assets             80
Short Term Debt                   1
Accounts Payable                 76
Total Current Liabilities        80
Long Term Debt                    0
Common Stock                     10
Retained Earnings                 5
Dividends per Share               0
Average Dividend Yield            8
Price to Book Ratio              14
Price Earnings Ratio (P/E)        3
Revenue/Basic Share               0
Inventory Turnover              185
Accounts Payable Turnover       126
Accounts Receivable Turnover

In [30]:
## estimating operating expenses by subtracting operating income or losses from revenue
alldata1.loc[alldata1['Total Operating Expenses'].isnull(), "Total Operating Expenses"] = alldata1["Revenue"]- alldata1["Operating Income or Losses"] 

In [31]:
## Handling Na values one by one
##We see that cost of revenue and gross profit na values are equal, delving further we see that these both are missing for same companies
gpna = alldata[alldata["Price Earnings Ratio (P/E)"].isna()]
gpna

,Year,Security,Revenue,Cost of Revenue,Gross Profit,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,...,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector
65,2018,CDAY,740.7,425.8,314.9,NaN,56.1,56.1,-0.53,112.7,...,0.0000,2.9773,NaN,6.4945,NaN,9.9139,11.3517,11.5845,4809.7585,Information Technology
200,2018,LIN,28084.0,16929.0,11155.0,NaN,2561.0,2561.0,3.11,4391.0,...,2.0019,NaN,NaN,50.5880,9.2030,4.4869,6.4141,94.2838,NaN,Materials
260,2018,NWSA,9024.0,4904.0,4120.0,NaN,248.0,248.0,-2.60,720.0,...,0.6666,0.9763,NaN,15.4865,16.7945,12.2660,6.2493,15.8763,9070.7590,Communication Services


In [32]:
gpna["Sector"].value_counts()
## Thus we can see assume that revenue and gross profit are missing for service sector, where COGS is usually zero because they are not producing 
## HEnce for financial sector we can update values equal to zero

Information Technology    1
Materials                 1
Communication Services    1
Name: Sector, dtype: int64

In [33]:
##Since we are already taking Operating expenses and income, we can drop cost of revenue and gross profit
alldata1 = alldata1.drop("Gross Profit", axis =1)
alldata1 = alldata1.drop("Cost of Revenue", axis = 1)


In [35]:
## Adressing Na values in Sector and historical market capitalization
gpna = alldata[alldata["Sector"].isna()]
## We observe that data hasn't captured sector for FICO and ZTS, manually inputing these
alldata1.loc[alldata1["Security"]=="FICO", "Sector"] = "Information Technology"

alldata1.loc[alldata1["Security"]=="ZTS", "Sector"] = "Health Care"


In [36]:
##Historical marketcap for Linde is missing since it merged in 2018, hence dropping that value
alldata1.dropna(subset=["Historical Market Cap"], inplace=True)


In [37]:
pd.set_option('display.max_columns', None)
gpna = alldata[alldata["Price to Book Ratio"].isna()]
gpna

,Year,Security,Revenue,Cost of Revenue,Gross Profit,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector
200,2018,LIN,28084.000,16929.000,11155.000,NaN,2561.000,2561.000,3.11,4391.000,4466.000,4255.000,93386.000,17272.000,5083.000,3219.000,12956.000,12288.000,1.0000,16529.000,3.30,2.0019,NaN,NaN,50.5880,9.2030,4.4869,6.4141,94.2838,NaN,Materials
245,2018,NRG,9478.000,7108.000,2370.000,8528.000,982.000,982.000,0.88,1496.000,563.000,1024.000,10628.000,3600.000,72.000,863.000,2398.000,6449.000,4.0000,-6022.000,0.12,0.3696,NaN,17.9333,31.1776,16.4347,9.1364,9.8524,-4.3504,11232.5415,Utilities
365,2018,SEE,4732.700,3230.600,1502.100,NaN,656.300,656.300,1.21,787.500,271.700,473.400,5050.200,1554.800,237.700,765.000,1488.600,3236.500,23.2000,1835.500,0.64,1.5683,NaN,13.9121,29.6907,6.1436,4.3911,9.2273,-2.2396,5422.9983,Materials
366,2019,SEE,4791.100,3226.300,1564.800,NaN,578.500,578.500,1.70,762.200,262.400,556.500,5765.200,1564.000,141.800,738.500,1436.200,3764.300,23.2000,1998.500,0.64,1.5357,NaN,14.0975,31.0506,5.7860,4.3255,9.3040,-1.2698,6154.2453,Materials
475,2018,VRSN,1214.969,192.134,1022.835,447.577,767.392,767.392,5.13,815.759,357.415,8.109,1914.504,1317.034,0.000,17.263,947.590,1785.047,0.3520,-17089.789,0.00,0.0000,NaN,33.4988,10.7091,NaN,13.8315,139.8365,-11.5421,17800.2867,Information Technology
476,2019,VRSN,1231.661,180.467,1051.194,425.534,806.127,806.127,5.17,852.457,508.196,17.884,1854.009,1278.589,4.632,15.907,965.166,1792.129,0.3532,-16477.490,0.00,0.0000,NaN,37.4644,10.3926,NaN,10.8813,94.7687,-12.7670,22488.6462,Information Technology
477,2020,VRSN,1265.052,180.177,1084.875,440.851,824.201,824.201,7.08,870.553,401.194,6.887,1766.910,1217.940,6.117,12.340,988.693,1795.242,0.3538,-15662.602,0.00,0.0000,NaN,30.7993,10.9949,NaN,12.7572,102.1398,-12.2517,24554.9080,Information Technology
478,2021,VRSN,1327.576,191.933,1135.643,460.773,866.803,866.803,7.01,914.743,223.487,6.265,1983.764,1268.735,2.485,8.997,1073.991,1788.194,0.3542,-14877.772,0.00,0.0000,NaN,36.3386,11.8518,NaN,17.9906,201.8820,-11.4055,28051.9326,Information Technology
479,2022,VRSN,1424.900,200.700,1224.200,481.800,943.100,943.100,6.24,990.000,373.600,7.600,1733.400,1038.700,0.000,9.800,1116.900,1789.500,0.3545,-14204.000,0.00,0.0000,NaN,32.9231,13.2057,NaN,21.3545,205.5391,-14.8357,21632.8320,Information Technology
520,2018,YUM,5688.000,1634.000,4054.000,3392.000,2296.000,2296.000,4.80,2433.000,292.000,561.000,4130.000,1207.000,321.000,202.000,1301.000,9751.000,0.0000,-7592.000,1.44,1.0688,NaN,19.6205,17.6646,NaN,10.1807,11.8377,-25.9020,28127.5200,Consumer Discretionary


In [38]:
alldata1.loc[alldata1['Price Earnings Ratio (P/E)'].isnull(), "Price Earnings Ratio (P/E)"] = (alldata1["Price to Book Ratio"]* alldata1["Book Value per Share"])/ alldata1["Basic Earnings per Share"] 

In [39]:
gpna = alldata1[alldata1["Price to Book Ratio"].isna()]
gpna["Book Value per Share"] > 0
## We notice that P/Bratio is nan for all the values where book value is less than zero, indicating negative P/b ratio,
## imputing these values from PE ratio
alldata1.loc[alldata1['Price to Book Ratio'].isnull(), "Price to Book Ratio"] = (alldata1["Price Earnings Ratio (P/E)"]*  alldata1["Basic Earnings per Share"])/ alldata1["Book Value per Share"] 

In [40]:
gpna = alldata1[alldata1["Average Dividend Yield"].isna()]
## We noticed that entries for which EBIT is missing EBITDA is also missing, so is accounts payable, inventoru turnover etc, indicating error in data collection
##hence deleting these values
gpna

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap
3,2018,AEP,16195.7,13513.0,2682.7,2682.7,3.90,5083.1,444.1,1926.5,68802.8,4113.9,3608.5,1874.3,8648.8,21648.2,3337.4000,9325.3,2.53,NaN,1.9334,18.9554,32.8663,6.1746,2.9213,8.4850,38.6578,36865.1968,Utilities,20533.1,20823.36,70.10,2.687200e+07
10,2018,BKR,22877.0,22176.0,701.0,701.0,0.46,2187.0,3723.0,5969.0,52439.0,14971.0,942.0,4025.0,9020.0,6285.0,0.1035,25.0,0.72,NaN,1.2741,31.8704,53.9663,4.1396,5.1348,3.8179,16.8744,22252.5000,Energy,20533.1,20823.36,70.10,2.687200e+07
34,2018,HON,41802.0,35097.0,6705.0,6705.0,9.10,7821.0,9287.0,6162.0,57773.0,24362.0,6458.0,5607.0,18924.0,9756.0,958.0000,33978.0,3.06,NaN,5.3022,14.3458,56.2611,6.4987,4.7181,6.2498,24.9178,96394.7520,Industrials,20533.1,20823.36,70.10,2.687200e+07
111,2019,AEP,15561.4,12969.1,2592.3,2592.3,3.89,5481.9,432.6,1705.1,75892.3,4077.8,4671.1,2085.8,10299.1,25861.4,3343.4000,9900.9,2.71,NaN,2.3738,22.3391,31.5203,4.8855,2.7041,8.5700,39.8145,46703.9567,Utilities,21381,21659.03,72.22,3.437586e+07
118,2019,BKR,23838.0,22764.0,1074.0,1074.0,0.23,2725.0,3249.0,6416.0,53369.0,15222.0,522.0,4268.0,10014.0,6942.0,0.1027,0.0,0.72,NaN,1.2003,56.3561,42.9514,4.2059,4.6772,3.8495,21.3525,26322.0100,Energy,21381,21659.03,72.22,3.437586e+07
142,2019,HON,36709.0,29858.0,6851.0,6851.0,8.52,8161.0,9067.0,6053.0,58679.0,24303.0,5063.0,5730.0,18098.0,11644.0,958.0000,37693.0,3.36,NaN,6.8057,19.3758,50.9140,5.5651,4.3105,6.0105,26.0076,125864.7000,Industrials,21381,21659.03,72.22,3.437586e+07
226,2020,BKR,20705.0,36683.0,-15978.0,-15978.0,-14.73,-14373.0,4132.0,5622.0,38007.0,16455.0,1107.0,3532.0,10227.0,7335.0,0.1035,-9942.0,0.72,NaN,1.6738,2.5769,30.6741,3.8777,4.4408,3.4399,12.4570,21579.7500,Energy,21060.5,21257.44,61.54,4.264076e+07
250,2020,HON,32637.0,26941.0,5696.0,5696.0,6.79,6912.0,14275.0,5440.0,64586.0,28175.0,6229.0,5750.0,19197.0,16983.0,958.0000,39905.0,3.63,NaN,8.4455,31.2625,46.3528,4.9762,3.8740,5.6795,25.1851,148209.3600,Industrials,21060.5,21257.44,61.54,4.264076e+07


In [58]:
alldata2 = alldata1

In [59]:
##There is mistake in data collection of average dividend yield, since these are important features, manually finding and inputing data
alldata2.loc[alldata1["Security"]=="BKR", "Average Dividend Yield"]  = [2.27, 2.88, 3.12, 2.96, 2.52]
alldata2.loc[alldata1["Security"]=="BKR", "Average Dividend Yield"]

10     2.27
118    2.88
226    3.12
334    2.96
442    2.52
Name: Average Dividend Yield, dtype: float64

In [60]:
alldata2.loc[alldata1["Security"]=="HON", "Average Dividend Yield"] =  [1.97, 4.95, 2.24, 1.64, 1.95]
#alldata2.loc[alldata1["Security"]=="BKR", "Average Dividend Yield"]

In [61]:
alldata2.loc[alldata1["Security"]=="AEP", "Average Dividend Yield"]  = [2.9, 2.88, 3.13, 3.36, 3.16]

In [62]:
alldata2 = alldata2.drop("EBIT", axis =1)
alldata2 = alldata2.drop("EBITDA", axis =1)

In [70]:
na_counts = alldata2.isna().sum()
na_counts

Year                            0
Security                        0
Revenue                         0
Total Operating Expenses        0
Operating Income or Losses      0
Basic Earnings per Share        0
Cash and Cash Equivalents       0
Accounts & Notes Receivable     0
Total Assets                    0
Total Current Assets            0
Short Term Debt                 0
Accounts Payable                0
Total Current Liabilities       0
Long Term Debt                  0
Common Stock                    0
Retained Earnings               0
Dividends per Share             0
Average Dividend Yield          0
Price to Book Ratio             0
Price Earnings Ratio (P/E)      0
Revenue/Basic Share             0
Accounts Payable Turnover       0
Accounts Receivable Turnover    0
Book Value per Share            0
Historical Market Cap           0
Sector                          0
GDP                             0
GNP                             0
confid_index                    0
Sp_mkt_cap    

In [64]:
alldata2 = alldata2.drop("Inventory Turnover", axis =1)

In [65]:
gpna = alldata1[alldata1["Retained Earnings"].isna()]
## We noticed that entries for which EBIT is missing EBITDA is also missing, so is accounts payable, inventoru turnover etc, indicating error in data collection
##hence deleting these values
gpna

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,EBIT,Basic Earnings per Share,EBITDA,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Inventory Turnover,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap
5,2018,AMT,7440.1,5535.1,1905.0,1905.0,2.79,4015.8,1208.7,459.0,33010.4,NaN,2754.8,1631.0,NaN,18405.1,4.5,NaN,3.15,1.8049,13.0753,56.9551,16.9245,NaN,NaN,NaN,12.0984,69771.2814,Real Estate,20533.1,20823.36,70.10,2.687200e+07
113,2019,AMT,7580.3,4891.9,2688.4,2688.4,4.27,5479.9,1501.2,462.2,42801.6,NaN,2928.2,1770.7,NaN,28132.1,4.5,NaN,3.78,1.5648,20.1339,53.5732,17.1376,NaN,NaN,NaN,11.4146,101784.9798,Real Estate,21381,21659.03,72.22,3.437586e+07
221,2020,AMT,8041.5,5154.0,2887.5,2887.5,3.81,5747.0,1746.3,511.6,47233.5,NaN,789.8,1935.2,NaN,35922.0,4.6,NaN,4.53,1.5725,24.3641,56.8117,18.1262,NaN,NaN,NaN,9.2127,99734.3118,Real Estate,21060.5,21257.44,61.54,4.264076e+07
329,2021,AMT,9356.9,6224.9,3132.0,3132.0,5.69,6579.7,1949.9,728.9,69887.9,NaN,4568.7,2582.0,NaN,47439.9,4.7,NaN,5.21,1.7649,26.2366,50.9203,20.7241,NaN,NaN,NaN,11.1486,133313.3100,Real Estate,23315.1,23488.17,65.15,5.376457e+07
437,2022,AMT,10711.1,8358.8,2352.3,2352.3,3.83,6930.2,2028.4,758.3,67194.5,NaN,4514.2,2569.1,NaN,42536.8,4.8,NaN,5.86,2.1935,17.7026,55.4482,23.2084,NaN,NaN,NaN,11.9677,98646.0413,Real Estate,25462.7,25640.13,63.10,4.106193e+07


In [67]:
alldata2.loc[alldata1["Security"]=="AMT", "Retained Earnings"] =  [-1200, -1017, -1343, -1142, -2102]

After manually cleaning and inputing data, next step is Average Value Imputation - Historic Company Values for remaining NA values

In [68]:
simimp = ["Accounts & Notes Receivable","Total Current Assets","Short Term Debt","Accounts Payable",
         "Total Current Liabilities","Common Stock","Accounts Payable Turnover", "Accounts Receivable Turnover"]

In [69]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
for col in simimp:
    vals = alldata2[col].values.reshape(-1,1)
    newvals = imp.fit_transform(vals).reshape(1,-1)
    alldata2[col] = newvals[0]

All data is clean with no more NAs, next step is feature engineering
Step 1: Dividend based features
Dividend yield will be used to feature dividend paid last year and distance(time in years) from last dividend payment. After that dividend per share and average dividend yield will be dropped s these cannot be known prior to dividend declaration and thus have no net predictive or applicable value in our pipeline.

In [71]:
alldata2["Average Dividend Yield"].apply(lambda s: 'PAID'  if s > 0 else 'UNPAID').value_counts(dropna=False)

PAID      423
UNPAID    116
Name: Average Dividend Yield, dtype: int64

In [72]:
#Dynamic payment duration variables
alldata2['Div_Paid?'] = alldata2["Average Dividend Yield"].apply(lambda s: 1 if s > 0 else 0)
#alldata2 = alldata2.drop("Paid_LastYr?", axis =1)

#When attempting to check whether the company has paid dividend in the previous year, we would have to drop the first recording for every firm since such data would be unavailable. However, since we already have limited data available from the API, it would  significantly reduce our training set and affect the applicability of the model. To get around it, we derive estimations for this first recording. As such, we assign a 1 (last dividend paid) if the company has paid a special majority of more dividends than not in recorded history, else 0. The reason we are stricter at a 0.67 requirement than 0.5 is to ensure greater certainty in the fundamental dividend features in our model

In [123]:
# sort the dataframe by date and firm
df = alldata2.sort_values(['Security', 'Year'])

In [124]:
df['prev_dividend'] = df.groupby('Security')['Div_Paid?'].shift()
df.loc[df['prev_dividend'] == 1, 'prev_year_dividend'] = 1

In [125]:
df

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,Basic Earnings per Share,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap,Div_Paid?,prev_dividend,prev_year_dividend
0,2018,AAPL,265595.0,30941.0,70898.0,3.0025,25913.0,23186.0,365725.0,131339.0,20748.0,55888.0,115929.0,93735.0,0.0475,70400.0,0.680,1.3297,10.0179,19.0155,13.3993,3.1039,12.9369,5.6334,1.073391e+06,Information Technology,20533.1,20823.36,70.10,2.687200e+07,1,NaN,NaN
108,2019,AAPL,260174.0,34462.0,63930.0,2.9900,48844.0,22926.0,338516.0,162819.0,16240.0,46236.0,105718.0,91807.0,0.0444,45898.0,0.750,1.4349,10.7447,18.4110,14.0853,3.1713,11.2844,5.0913,9.722690e+05,Information Technology,21381,21659.03,72.22,3.437586e+07,1,1.0,1.0
216,2020,AAPL,274515.0,38668.0,66288.0,3.3100,38016.0,16120.0,323888.0,143713.0,15229.0,42296.0,105392.0,107049.0,0.1698,14966.0,0.795,0.9743,29.1732,34.2703,15.8203,3.8294,14.0611,3.8487,1.906151e+06,Information Technology,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1.0
324,2021,AAPL,365817.0,43887.0,108949.0,5.6700,34940.0,26278.0,351002.0,134836.0,17141.0,54763.0,125481.0,119381.0,0.1643,5562.0,0.865,0.6114,38.2537,26.2512,21.9035,4.4406,17.2563,3.8407,2.413423e+06,Information Technology,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0
432,2022,AAPL,394328.0,51345.0,119437.0,6.1500,23646.0,28184.0,352755.0,135405.0,22773.0,64115.0,153982.0,109707.0,0.1594,-3068.0,0.900,0.5488,47.3313,24.6203,24.3173,3.7334,14.4808,3.1782,2.398369e+06,Information Technology,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2020,ZTS,6675.0,4431.0,2244.0,3.4400,3604.0,1013.0,13609.0,6611.0,644.0,457.0,2170.0,6758.0,5.0000,5659.0,0.850,0.4849,20.8716,43.2166,14.0378,6.0026,6.3602,7.9294,7.866509e+04,Health Care,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1.0
430,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0
431,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0
538,2022,ZTS,8080.0,5163.0,2917.0,4.5100,3581.0,1215.0,14925.0,7506.0,1395.0,405.0,3167.0,6738.0,5.0000,8668.0,1.350,0.6367,15.4304,29.8280,17.2321,6.8395,6.8825,9.4975,6.797107e+04,Health Care,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1.0


In [126]:
def fill_na(group):
    if group.dropna().sum() > 3:
        group.fillna(1, inplace=True)
    else:
        group.fillna(0, inplace=True)
    return group

In [127]:
# apply the function to each group of the dataframe
df['prev_year_dividend'] = df.groupby('Security')['prev_year_dividend'].apply(fill_na)

In [128]:
#We now define a categorical variable - distance from last dividend. It can either be 1-5 or UNKNOWN. The reason this is being treated
#as discrete not continuous is to allow the fact that since companies have existed for long durations, imputing unavailable data may lead
#to excessive engineering of the training set, not being representative of the original data itself.
dist = df['prev_year_dividend'].map({1:0, 0:1})
dist = np.cumsum(dist)
dist = dist - dist.where(df.Security != df.Security.shift(-1)).shift(1).ffill().fillna(0)
#dist.tail(20)
#dist

In [129]:
df['DistanceFromLast'] = dist + 1

df['DistanceFromLast'] = df['DistanceFromLast'].astype('int').astype('str')
df.loc[df.Security != df.Security.shift(1), "DistanceFromLast"] = 'UNKNOWN'

df

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,Basic Earnings per Share,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap,Div_Paid?,prev_dividend,prev_year_dividend,DistanceFromLast
0,2018,AAPL,265595.0,30941.0,70898.0,3.0025,25913.0,23186.0,365725.0,131339.0,20748.0,55888.0,115929.0,93735.0,0.0475,70400.0,0.680,1.3297,10.0179,19.0155,13.3993,3.1039,12.9369,5.6334,1.073391e+06,Information Technology,20533.1,20823.36,70.10,2.687200e+07,1,NaN,1.0,UNKNOWN
108,2019,AAPL,260174.0,34462.0,63930.0,2.9900,48844.0,22926.0,338516.0,162819.0,16240.0,46236.0,105718.0,91807.0,0.0444,45898.0,0.750,1.4349,10.7447,18.4110,14.0853,3.1713,11.2844,5.0913,9.722690e+05,Information Technology,21381,21659.03,72.22,3.437586e+07,1,1.0,1.0,1
216,2020,AAPL,274515.0,38668.0,66288.0,3.3100,38016.0,16120.0,323888.0,143713.0,15229.0,42296.0,105392.0,107049.0,0.1698,14966.0,0.795,0.9743,29.1732,34.2703,15.8203,3.8294,14.0611,3.8487,1.906151e+06,Information Technology,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1.0,1
324,2021,AAPL,365817.0,43887.0,108949.0,5.6700,34940.0,26278.0,351002.0,134836.0,17141.0,54763.0,125481.0,119381.0,0.1643,5562.0,0.865,0.6114,38.2537,26.2512,21.9035,4.4406,17.2563,3.8407,2.413423e+06,Information Technology,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0,1
432,2022,AAPL,394328.0,51345.0,119437.0,6.1500,23646.0,28184.0,352755.0,135405.0,22773.0,64115.0,153982.0,109707.0,0.1594,-3068.0,0.900,0.5488,47.3313,24.6203,24.3173,3.7334,14.4808,3.1782,2.398369e+06,Information Technology,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2020,ZTS,6675.0,4431.0,2244.0,3.4400,3604.0,1013.0,13609.0,6611.0,644.0,457.0,2170.0,6758.0,5.0000,5659.0,0.850,0.4849,20.8716,43.2166,14.0378,6.0026,6.3602,7.9294,7.866509e+04,Health Care,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1.0,1
430,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0,1
431,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1.0,1
538,2022,ZTS,8080.0,5163.0,2917.0,4.5100,3581.0,1215.0,14925.0,7506.0,1395.0,405.0,3167.0,6738.0,5.0000,8668.0,1.350,0.6367,15.4304,29.8280,17.2321,6.8395,6.8825,9.4975,6.797107e+04,Health Care,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1.0,1


In [130]:
df['DistanceFromLast'].value_counts()

1          333
UNKNOWN    107
3           27
4           23
5           23
6           21
2            5
Name: DistanceFromLast, dtype: int64

In [131]:
df.loc[df['DistanceFromLast'] == "6"] 

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,Basic Earnings per Share,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap,Div_Paid?,prev_dividend,prev_year_dividend,DistanceFromLast
433,2022,ACGL,9614.808,8126.315,1618.759,3.90,855.118,12090.897,47990.507,4824.000,31.861,855.000,3531.000,2845.439,0.6540,15892.065,0.0,0.0,1.924700,13.0157,26.0838,8.3792,7.6360,32.6183,2.325032e+04,Financials,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
441,2022,BIO,2802.249,2319.633,482.616,-121.79,434.215,494.645,13501.666,3157.976,36.801,135.041,568.708,1351.313,0.0030,9898.203,0.0,0.0,1.294300,30.2994,94.0826,9.9790,6.1039,324.8866,1.244470e+04,Health Care,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
445,2022,CDAY,1246.200,1272.000,-25.800,-0.48,431.900,180.100,7917.200,4894.000,18.800,54.300,4533.500,1237.100,1.5000,-372.600,0.0,0.0,4.679000,730.0830,8.1483,14.5849,7.6360,13.7102,9.869904e+03,Information Technology,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
446,2022,CDNS,3561.718,2488.032,1073.686,3.13,882.325,314.666,5137.071,1706.767,136.737,855.000,1347.696,787.415,3.2916,3895.240,0.0,0.0,15.956500,49.2282,13.1333,8.3792,14.2393,10.0673,4.380251e+04,Information Technology,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
451,2022,CNC,144547.000,143229.000,1318.000,2.09,12074.000,13272.000,76870.000,30128.000,262.000,16745.000,28464.000,21071.000,1.0000,9341.000,0.0,0.0,1.877500,14.0460,251.3026,7.6521,11.3326,43.6801,4.516734e+04,Health Care,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
453,2022,CSGP,2182.399,1317.442,450.949,0.93,4967.970,153.945,8402.470,5185.867,36.049,28.460,372.615,1069.531,4.0660,1829.619,0.0,0.0,4.574500,81.0277,5.5072,16.3304,15.6578,16.8936,3.142753e+04,Industrials,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
458,2022,EPAM,4824.698,4251.732,572.966,7.32,1681.344,932.626,4009.151,2759.625,43.213,30.852,747.483,150.010,0.0580,2248.948,0.0,0.0,6.295400,40.4660,84.2139,118.0159,5.6709,52.0602,1.889585e+04,Information Technology,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
461,2022,FSLR,2619.319,350.605,-27.236,-0.41,1481.269,324.337,8251.228,3791.421,9.193,341.409,1038.048,224.938,0.1070,3140.289,0.0,0.0,2.736300,24.6916,24.5828,9.3666,6.9499,54.7426,1.596898e+04,Information Technology,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
463,2022,GOOGL,282836.000,207994.000,74842.000,4.59,21879.000,40258.000,365264.000,164795.000,2775.000,5128.000,69300.000,27202.000,13.2420,195563.000,0.0,0.0,4.425900,18.1407,21.6517,22.8756,7.1098,19.9349,1.133667e+06,Communication Services,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6
465,2022,HOLX,4862.800,1357.200,1640.200,5.18,2339.500,617.600,9071.200,3862.000,41.400,197.700,937.700,2880.200,3.0000,1600.300,0.0,0.0,3.191900,12.6572,19.3331,9.6127,6.2332,19.7312,1.556437e+04,Health Care,25462.7,25640.13,63.1,4.106193e+07,0,0.0,0.0,6


In [132]:
df.loc[df['Security'] == "FICO"] 

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,Basic Earnings per Share,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap,Div_Paid?,prev_dividend,prev_year_dividend,DistanceFromLast
28,2018,FICO,1000.146,824.787,175.359,4.26,90.023,266.742,1330.467,396.389,235.000,20.251,473.903,528.944,0.290,1764.524,0.0,0.0227,23.0707,64.5075,33.6625,15.7389,4.5949,9.9065,6631.3783,Information Technology,20533.1,20823.36,70.10,2.687200e+07,1,NaN,0.0,UNKNOWN
136,2019,FICO,1160.083,906.535,253.548,6.63,106.426,297.427,1433.448,455.706,218.000,23.118,490.828,606.790,0.289,1956.648,0.0,0.0000,30.3177,56.7234,40.0305,15.5339,4.1125,10.0113,8785.0829,Information Technology,21381,21659.03,72.22,3.437586e+07,0,1.0,1.0,2
244,2020,FICO,1294.562,998.593,295.969,8.13,157.394,334.180,1606.240,534.078,119.973,23.033,414.511,815.718,0.291,2193.059,0.0,0.0000,37.3831,57.9835,44.5372,15.6505,4.0993,11.3790,12376.8565,Information Technology,21060.5,21257.44,61.54,4.264076e+07,0,0.0,0.0,3
352,2021,FICO,1316.536,811.047,505.489,13.65,195.354,312.107,1567.776,550.974,272.074,20.749,559.207,1062.688,0.276,2585.143,0.0,0.0000,37.3831,39.7384,45.8181,15.1872,4.0742,-4.0243,10970.1342,Information Technology,23315.1,23488.17,65.15,5.376457e+07,0,0.0,0.0,4
460,2022,FICO,1377.270,834.856,542.414,14.34,133.202,322.410,1442.034,484.715,49.369,17.273,331.459,1862.861,0.252,2958.684,0.0,0.0000,37.3831,29.1966,52.8865,15.8947,4.3412,-31.8815,10363.6995,Information Technology,25462.7,25640.13,63.10,4.106193e+07,0,0.0,0.0,5


In [133]:
df = df.drop("prev_dividend", axis =1)

In [163]:
# Assuming the dataframe is named 'df' and has columns 'sector', 'year', 'firm', and 'dividend paid'

# Group the data by sector and year, and count the number of firms that paid dividends
dividend_counts = df.groupby(['Sector', 'Year'])['prev_year_dividend'].sum()



# Group the data by sector and year, and count the number of unique firms
firm_counts = df.groupby(['Sector', 'Year'])['Security'].nunique()

# Print the result
#print(firm_counts)

sec_prop = dividend_counts/firm_counts
sec_prop = pd.DataFrame(sec_prop)
sec_prop.columns = [ "SecProp"]
sec_prop["Year"]
#sec_prop["Sectorprop"] = 

SecProp
Sector                 Year          
Communication Services 2018  0.571429
                       2019  0.571429
                       2020  0.571429
                       2021  0.571429
                       2022  0.571429
Consumer Discretionary 2018  1.000000
                       2019  1.000000
                       2020  1.000000
                       2021  1.000000
                       2022  1.000000
Consumer Staples       2018  1.000000
                       2019  1.000000
                       2020  1.000000
                       2021  1.000000
                       2022  1.000000
Energy                 2018  1.000000
                       2019  1.000000
                       2020  1.000000
                       2021  1.000000
                       2022  1.000000
Financials             2018  0.928571
                       2019  0.928571
                       2020  0.928571
                       2021  0.928571
                       2022  0.928571
Health Care            2018  0.666667
                       2019  0.666667
                       2020  0.666667
                       2021  0.666667
                       2022  0.666667
Industrials            2018  0.800000
                       2019  0.800000
                       2020  0.866667
                       2021  0.866667
                       2022  0.866667
Information Technology 2018  0.520000
                       2019  0.560000
                       2020  0.520000
                       2021  0.520000
                       2022  0.520000
Materials              2018  1.000000
                       2019  0.800000
                       2020  1.000000
                       2021  1.000000
                       2022  1.000000
Real Estate            2018  1.000000
                       2019  1.000000
                       2020  1.000000
                       2021  1.000000
                       2022  1.000000
Utilities              2018  0.857143
                       2019  1.000000
                       2020  0.857143
                       2021  0.857143
                       2022  0.857143

In [164]:
df = df.merge(sec_prop, on=['Sector', 'Year'], how='left')

In [165]:
df

,Year,Security,Revenue,Total Operating Expenses,Operating Income or Losses,Basic Earnings per Share,Cash and Cash Equivalents,Accounts & Notes Receivable,Total Assets,Total Current Assets,Short Term Debt,Accounts Payable,Total Current Liabilities,Long Term Debt,Common Stock,Retained Earnings,Dividends per Share,Average Dividend Yield,Price to Book Ratio,Price Earnings Ratio (P/E),Revenue/Basic Share,Accounts Payable Turnover,Accounts Receivable Turnover,Book Value per Share,Historical Market Cap,Sector,GDP,GNP,confid_index,Sp_mkt_cap,Div_Paid?,prev_year_dividend,DistanceFromLast,SecProp
0,2018,AAPL,265595.0,30941.0,70898.0,3.0025,25913.0,23186.0,365725.0,131339.0,20748.0,55888.0,115929.0,93735.0,0.0475,70400.0,0.680,1.3297,10.0179,19.0155,13.3993,3.1039,12.9369,5.6334,1.073391e+06,Information Technology,20533.1,20823.36,70.10,2.687200e+07,1,1.0,UNKNOWN,0.520000
1,2019,AAPL,260174.0,34462.0,63930.0,2.9900,48844.0,22926.0,338516.0,162819.0,16240.0,46236.0,105718.0,91807.0,0.0444,45898.0,0.750,1.4349,10.7447,18.4110,14.0853,3.1713,11.2844,5.0913,9.722690e+05,Information Technology,21381,21659.03,72.22,3.437586e+07,1,1.0,1,0.560000
2,2020,AAPL,274515.0,38668.0,66288.0,3.3100,38016.0,16120.0,323888.0,143713.0,15229.0,42296.0,105392.0,107049.0,0.1698,14966.0,0.795,0.9743,29.1732,34.2703,15.8203,3.8294,14.0611,3.8487,1.906151e+06,Information Technology,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1,0.520000
3,2021,AAPL,365817.0,43887.0,108949.0,5.6700,34940.0,26278.0,351002.0,134836.0,17141.0,54763.0,125481.0,119381.0,0.1643,5562.0,0.865,0.6114,38.2537,26.2512,21.9035,4.4406,17.2563,3.8407,2.413423e+06,Information Technology,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1,0.520000
4,2022,AAPL,394328.0,51345.0,119437.0,6.1500,23646.0,28184.0,352755.0,135405.0,22773.0,64115.0,153982.0,109707.0,0.1594,-3068.0,0.900,0.5488,47.3313,24.6203,24.3173,3.7334,14.4808,3.1782,2.398369e+06,Information Technology,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1,0.520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,2020,ZTS,6675.0,4431.0,2244.0,3.4400,3604.0,1013.0,13609.0,6611.0,644.0,457.0,2170.0,6758.0,5.0000,5659.0,0.850,0.4849,20.8716,43.2166,14.0378,6.0026,6.3602,7.9294,7.866509e+04,Health Care,21060.5,21257.44,61.54,4.264076e+07,1,1.0,1,0.666667
535,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1,0.666667
536,2021,ZTS,7776.0,5016.0,2760.0,4.2900,3485.0,1133.0,13900.0,6930.0,41.0,436.0,1797.0,6743.0,5.0000,7186.0,1.075,0.4590,25.3846,51.6928,16.3930,5.8186,7.2470,9.6133,1.153223e+05,Health Care,23315.1,23488.17,65.15,5.376457e+07,1,1.0,1,0.666667
537,2022,ZTS,8080.0,5163.0,2917.0,4.5100,3581.0,1215.0,14925.0,7506.0,1395.0,405.0,3167.0,6738.0,5.0000,8668.0,1.350,0.6367,15.4304,29.8280,17.2321,6.8395,6.8825,9.4975,6.797107e+04,Health Care,25462.7,25640.13,63.10,4.106193e+07,1,1.0,1,0.666667


In [166]:
# All data is clean and new features have been engineered as well, deleting Div_paid column and saving the data in csv
df = df.drop("Div_Paid?", axis =1)

In [167]:
df.to_csv("dividenddata.csv")